In [ ]:
import os
from os import listdir
from os.path import isfile, join

import re

import numpy as np

from datetime import datetime

import json

import pandas as pd

from IPython.display import display, Markdown

In [ ]:
import httpimport

with httpimport.remote_repo(['visualizador_dataframes'],'https://bonafe.github.io/CienciaDadosPython/src/componentes/'):
    from visualizador_dataframes import VisualizadorDataFrames

In [ ]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 10000)

In [ ]:
def prefixoData(data):
    return data.strftime("%Y_%m_%d")

In [ ]:
pasta_capacitacao = os.path.dirname("z:/bases_capacita/projeto_capacita/sa3/entrega_2022/")
todos_arquivos_pasta_capacitacao = listdir(pasta_capacitacao)

print ('\n*************** Arquivos Base Capacitação ***************\n\n')
print (*todos_arquivos_pasta_capacitacao, sep = '\n')

In [ ]:
df_conteudo_programatico = pd.read_csv( os.path.join(pasta_capacitacao,"2022_06_01-conteudo_programatico_acoes_capacitacao.csv"), index_col=None, header=0)
df_conteudo_programatico.name = "conteudo_programatico"

In [ ]:

visualizador = VisualizadorDataFrames(df_conteudo_programatico)


In [ ]:
visualizador.exibirMetricas("conteudo_programatico", log=False)

In [ ]:
df_conteudo_programatico.info()

In [ ]:
display("Número de registos conteúdo programático:")
display(df_conteudo_programatico["id_acao"].count())
display("Número de ações conteúdo programático:")
display(df_conteudo_programatico["id_acao"].nunique())


In [ ]:
df_conteudo_programatico.value_counts(["id_acao","id_trimestre"])

In [ ]:
#Cria uma coluna id_acao_trimestre que representa um conjunto de conteúdos programáticos 
df_conteudo_programatico["id_acao_trimestre"] = (
    df_conteudo_programatico["id_acao"].astype(str) + 
    "_" + 
    df_conteudo_programatico["id_trimestre"].astype(str))

In [ ]:
display(f'Número de conjuntos de conteúdo programátrico (id_acao + id_trimestre): {df_conteudo_programatico["id_acao_trimestre"].nunique()}')
display(Markdown("### Cada conjunto (id_acao + id_trimestre) possue vários registros de conteúdo numerados em sequencia"))

In [ ]:
#TODO: descobrir um jeito de não ter que iterar pelas linhas
#df_agrupado = df.groupby(['id_acao_trismestre','sequencia_conteudo'])['conteudo_programatico'].apply(lambda x: ','.join(x)).reset_index()

#Ordena o dataframe de conteúdo programático por conjunto de conteúdos e a sequencia do conteúdo
df_conteudo_programatico.sort_values(['id_acao_trimestre', 'sequencia_conteudo'], ascending=[True, False], inplace=True)

df_iter = df_conteudo_programatico.reset_index()

lista = []
obj ={
    'id_acao_trimestre': None
}
id_atual = None

#Percorre o dataframe concatenando os diversos conteúdos programáticos de um conjunto (id_acao e id_trimestre)
for index, row in df_iter.iterrows():
    
    
    #Se mudou de ação e existe ação anterior
    if row["id_acao_trimestre"] != obj["id_acao_trimestre"]:
        if obj["id_acao_trimestre"] != None:
            lista.append(obj)        
            
        obj = {
            'id_acao':row["id_acao"],
            'id_trimestre':row["id_trimestre"],
            'id_acao_trimestre':row["id_acao_trimestre"],
            'titulo_acao' : row["titulo_acao"],
            'conteudo_programatico':row["conteudo_programatico"]
        }
    else:
        obj['conteudo_programatico'] += " " + row["conteudo_programatico"]
    
df_conteudo_programatico_agrupado = pd.DataFrame(lista)
df_conteudo_programatico_agrupado.name = "conteudo_programatico_agrupado"

In [ ]:
visualizador = VisualizadorDataFrames(df_conteudo_programatico_agrupado)

visualizador.exibirMetricas("conteudo_programatico_agrupado", log=False)

In [ ]:
#Apenas pode haver um item por conjunto id_acao / id_trimestre --- OK
df_conteudo_programatico_agrupado.value_counts(["id_acao","id_trimestre"])

In [ ]:
df_conteudo_programatico_agrupado.count()

In [ ]:
pasta_destino = f'z:/bases_capacita/projeto_capacita/tcc_bonafe/'
nome_arquivo = f'{pasta_destino}/{prefixoData(datetime.now())}-conteudo_programatico_agrupado.csv'

In [ ]:
df_conteudo_programatico_agrupado.info()

In [ ]:
df_conteudo_programatico_agrupado.to_csv (nome_arquivo, index=False)